# Monte Carlo Parte 3


Nesta parte tentaremos resolver o problema de calcular a energia de um sistema atômico multidimensional. Lembrando que nas partes 1 e 2 resolvemos problemas considerando o sistema como unidimensional e verificamos que a solução de integrais unidimensionais como o método Monte Carlo não é tão vantajoso quando comparado com métodos numéricos clássicos, tais como o método do trapézio composto.

- O problema que devemos resolver é:

   Considere um sistema formado por 100 átomos de Argônio, em uma temperatura $T = 90 K$, pressão $p = 20 atm$ e densidade do argônio d = 1,394 $g/cm³$. A partir do potencial de interação de LJ e utilizando uma distribuição uniforme. Calcule a energia interna média deste sistema.
   
Para resolver esse problema vamos considerá-lo em partes.

### Primeira Parte

Vamos estabelecer que os 100 átomos de Argônio estão limitados em uma região compreendida por uma caixa de dimensões cúbicas. Assim deveremos calcular os lados desta caixa a partir da quantidade de átomos que estamos colocando na caixa e sua densidade.

As relações usadas para o tamanho dos lados da caixa são:

1) calcular a quantidade de matéria (n) relacionada com N = 100 átomos.

$$\frac{1 mol (Ar)}{6,02 \times 10^{23} átomos} = \frac{n (Ar)}{100 átomos}$$

$$n = 1,661 \times 10^{-22} mol (Ar)$$

2) calcular a massa (m) relacionada com a quantidade de matéria (n) através da massa Molar do argônio $MM(Ar) = 39,963 \frac{g}{mol}$.

$$\frac{39,963g}{1 mol (Ar)} = \frac{m}{1,661 \times 10^{-22} mol (Ar)}$$

$$m = 6,6383 \times 10^{-21}g$$

3) Como o volume de uma caixa cúbica é $v = L^3$ e a densidade é $d = m/v$, então o lado da caixa é:

$$L = \Big(\frac{m}{d}\Big)^{\frac{1}{3}}.$$

4) Substituindo o valor da massa e a densidade do Argônio, teremos: 

$$L = 1,682 \times 10^{-7} cm$$

5) Convertendo para angstrom temos:

$$L = 16,82 angstrom$$

Sabendo esses passos podemos criar uma função em python que calcula o lado da caixa de dimensões cúbicas.

In [1]:
def lado_cubo(N, MM, d):
    '''
    Essa função recebe como parâmetros:
    o número de átomos do sistema, a
    massa molar do átomo e a densidade.
    Retornando o lado da caixa de dimensões
    cúbicas.
    
    Unidades de Medida (entrada)
    ------------------
    número de átomos(N)   [sem unidades]
    Massa Molar(MM)       [g/mol]
    densidade(d)          [g/cm^3]
    
    Unidades de Medida (saida/retorno)
    ------------------
    lado(l)    [angstrom]
    '''
    
    Na = 6.02*(10**23)
    quantidade_materia = N / Na
    massa = MM * quantidade_materia
    
    lado = (massa/d)**(1/3)
    lado_metros = lado/100
    lado_angstrom = lado_metros * (10**10)
    
    return lado_angstrom
    

In [2]:
# Vamos passar as informações do problema e verificar
#se o função lado_cubo retorna o valor de 16,82 angstrom.

numeros_de_atomos = 100
massa_molar = 39.963
densidade = 1.394
print(lado_cubo(numeros_de_atomos, massa_molar, densidade))

16.824142545641326


Note que o valor de retorno da função lado_cubo() foi exatamente o que esperavamos, assim temos a primeira parte concluida :).

### Segunda parte

Nesta parte vamos criar uma função que gere os valores das coordenadas x, y e z da partícula de maneira aleatória  e retorne o valor da coordenada espacial $r$ que está contida no interior da caixa de dimensões cúbicas. Podemos descrever o valor de $r$ como sendo:

$$r = (x² + y² + z²)^{\frac{1}{2}}$$

In [26]:
import random


def coords_x_y_z(l):
    '''
    Essa função recebe como parâmetro
    apenas o valor do lado da caixa de
    dimensões cúbicas. Retornando a 
    posição x, y, z de uma partícula no
    interior da caixa.    
    '''
    
    minimo = -l/2
    maximo = l/2
    
    x = random.uniform(minimo, maximo)
    y = random.uniform(minimo, maximo)
    z = random.uniform(minimo, maximo)
    
    r = (x**2 + y**2 + z**2)**(1/2)
    
    return r


# Teste da função coords_x_y_z(l)

numeros_de_atomos = 100
massa_molar = 39.963
densidade = 1.394
lado = lado_cubo(numeros_de_atomos, massa_molar, densidade)

print(coords_x_y_z(lado))
    
    

9.424488957019301


Executando a função coords_x_y_z() percebemos que as coordenadas geradas produzem um valor menor que o valor do lado do cubo, portanto podemos entender que o ponto para esses coordenadas está de fato no interior da caixa. Seria interessante executarmos a função coords_x_y_z() várias vezes para verificarmos se não existe algum valor fora do intervalo de interesse.


In [27]:
for i in range(0, 501):
    print(f'Lado={round(coords_x_y_z(lado), 2)} no passo i={i}')

Lado=8.63 no passo i=0
Lado=8.54 no passo i=1
Lado=9.53 no passo i=2
Lado=7.22 no passo i=3
Lado=9.97 no passo i=4
Lado=3.53 no passo i=5
Lado=13.68 no passo i=6
Lado=10.62 no passo i=7
Lado=4.6 no passo i=8
Lado=7.51 no passo i=9
Lado=11.75 no passo i=10
Lado=6.06 no passo i=11
Lado=10.14 no passo i=12
Lado=3.67 no passo i=13
Lado=10.46 no passo i=14
Lado=7.32 no passo i=15
Lado=7.77 no passo i=16
Lado=4.74 no passo i=17
Lado=6.08 no passo i=18
Lado=3.39 no passo i=19
Lado=8.37 no passo i=20
Lado=4.12 no passo i=21
Lado=8.24 no passo i=22
Lado=11.21 no passo i=23
Lado=9.58 no passo i=24
Lado=5.14 no passo i=25
Lado=6.67 no passo i=26
Lado=7.75 no passo i=27
Lado=11.38 no passo i=28
Lado=9.37 no passo i=29
Lado=8.89 no passo i=30
Lado=9.12 no passo i=31
Lado=10.15 no passo i=32
Lado=6.43 no passo i=33
Lado=4.66 no passo i=34
Lado=7.78 no passo i=35
Lado=6.49 no passo i=36
Lado=11.71 no passo i=37
Lado=5.83 no passo i=38
Lado=4.39 no passo i=39
Lado=8.91 no passo i=40
Lado=6.26 no passo

### Terceira Parte

Agora que já temos como calcular as coordenas das partículas no interior da caixa cúbica de forma aleatória podemos calcular o valor da interação de pares considerando o potencial de LJ. A estrutura funcional do potencial de LJ é

$$U(r) = 4 \epsilon \Biggl[\Biggl(\frac{\sigma}{(x² + y² + z²)^{\frac{1}{2}}}\Biggl)^{12}-\Biggl(\frac{\sigma}{(x² + y² + z²)^{\frac{1}{2}}}\Biggl)^{6} \Biggl]$$

Portanto deveremos implementar a função acima para calcular o potencial de interação.

In [32]:
def potencial(r, e, si):
    '''
    Essa função é o potencial de LJ, que 
    recebe como parâmetros de entrada as 
    posições aleatórias que foram geradas 
    a partir das dimensões de uma caixa 
    cúbica, a energia de interação epsilon
    e a distância de interação sigma.
    Retorna o potencial de interação.
    
    Unidades de Medida (entrada)
    ------------------
    distancia(r)          [angstrom]
    epsilon(e)            [kcal/mol]
    sigma(si)             [angstrom]
    
    Unidades de Medida (saida/retorno)
    ------------------
    potencial(U)          [kcal/mol]    
    '''
    
    U = 4*e*((si/r)**12 - (si/r)**6)
    return U

# Teste da função potencial(r, e, si)

numeros_de_atomos = 100
massa_molar = 39.963
densidade = 1.394
lado = lado_cubo(numeros_de_atomos, massa_molar, densidade)


r = coords_x_y_z(lado)
epsilon =  0.2378
sigma = 3.41

U = potencial(r, epsilon, sigma)
print(U)
    

-0.0017923933732648617


#### Quarta Parte

Agora devemos somar as energias potenciais correspondentes a todas as partículas que fazem parte do sistema de interesse. Neste caso sabemos que:

$$U(r) = \sum_{i}\sum_{j}U(r_{ij}) = \sum_{i}\sum_{j}U_{ij}(r).$$

In [46]:
# Somando as energias potenciais

numero_de_atomos = 100
massa_molar = 39.963
densidade = 1.394

lado = lado_cubo(numeros_de_atomos, massa_molar, densidade)

epsilon =  0.2378
sigma = 3.41

U_positivo = 0
U_total = 0
for i in range(numero_de_atomos):
    r = coords_x_y_z(lado)
    U = potencial(r, epsilon, sigma)
    U_total += U
    
    if U > 0:
        U_positivo += 1   
      
    print(U)


print('\n')    
print(f'Energias Positivas: {U_positivo} de {numero_de_atomos}')
print('------')    
print(f'Energia Total: {U_total}')    

-0.0227923365432711
-0.004986633206711645
-0.0011996436770703816
-0.14423735072239946
-0.01595316133166694
-0.037236812416047683
-0.0009538369613314518
-0.014736408213618395
-0.09653182400912894
-0.0031179318627665127
-0.01059875549337487
-0.0007857141204225915
-0.0017335744239872237
2.0098865298532176
-0.00038310558824759935
-0.0007142804201843185
-0.006210492168315925
-0.07335011248397122
-0.011989412554353396
-0.0010807611644981748
-0.0021688380820901187
-0.008726396486802795
-0.0024619255034511074
-0.0011359321793199529
-0.00598918893976587
-0.039456062815864515
-0.009277119320429222
-0.047263729866941405
-0.033842484382900155
-0.0008276290972772875
-0.0009498990838133469
-0.0003824456502123388
-0.0004939392196519101
-0.0007793062633673774
-0.003282688618229197
-0.0070966183975577695
-0.0041501904474977236
-0.0036676953285808144
-0.0022598236909664718
-0.0062363471078649025
-0.07103028497343139
-0.01332352827031411
-0.0014799625737468285
-0.0016976654569146002
-0.02559295794938306


### Quinta parte

Agora será necessário fazer uma discussão para calcular o valor médio da energia interna do sistema. Como sabemos a termodinâmica trabalha com valores médios. Portanto o valor médio de uma propriedade do sistema é dada por:

$$ \langle f \rangle = \sum_{i} f_{i}P_{i}.$$

Pela distribuição de Boltzmann, 

$$P_{i} = \frac{e^{-\beta E_i}}{\sum_{i}e^{-\beta E_i}}.$$

Logo o valor médio de uma propriedade é dado na forma:

$$\langle f \rangle = \frac{\sum_{i} f_{i} e^{-\beta E_i}}{\sum_{i}e^{-\beta E_i}}.$$

Como nosso sistema é composto por um enorme conjunto de partículas, então a propriedade média discreta $\langle f \rangle$ pode ser substituida por uma propriedade contínua, assumindo que existem também uma coleção enorme de estados $E_i$ no sistema de interesse. Logo $E_i$ pode ser entendido como a energia total do sistema e sabemos que a energia total é descrita pela seguinte Hamiltoniana:

$$H(p,r) = k(p) + U(r),$$

em que a energia cinética é dada na forma $k(p) = \frac{p²}{2m}$, enquanto o energia potencial $U(r)$ pode assumir as mais diversas formas. Assumindo um forma contínua da propriedade $f$ teremos:

$$\langle f \rangle = \frac{\int_{p}\int_{r} f(p, r) e^{-\beta H(p,r)} dp dr}{\int_{p}\int_{r} e^{-\beta H(p,r)} dp dr}.$$

A equação acima descreve a forma de calcularmos o valor de uma propriedade média a partir da energia total do sistema.

No caso de buscarmos calcular a energia média do sistema devemos considerar que:

$$\langle E \rangle = - \frac{\partial ln Z}{\partial \beta},$$

sabendo que a função de partição $Z$ é da forma

$$Z = \int_{p}\int_{r} e^{-\beta H(p,r)} dp dr.$$

Expandindo a função de partição em termo da Hamiltoniana temos,

$$Z = \int_{p}\int_{r} e^{-\beta (k(p) + U(r))} dp dr$$

$$Z = \int_{p} e^{-\beta k(p)} dp \int_{r} e^{-\beta U(r)} dr,$$ 

como sabemos o valor da função energia cinética, também sabemos a solução da integral que envolve somente as coordenadas do momento na energia cinética,

$$\int_{p} e^{-\beta k(p)} dp = \Big(\frac{2m\pi}{\beta}\Big)^{\frac{3N}{2}}.$$

Portanto a função de partição é da forma:

$$Z = \Big(\frac{2m\pi}{\beta}\Big)^{\frac{3N}{2}} \int_{r} e^{-\beta U(r)} dr,$$

aplicando o logaritmo natural a função de partição teremos, 

$$ln(Z) = \frac{3N}{2} (2m\pi) - \frac{3N}{2} \beta + ln\Big(\int_{r} e^{-\beta U(r)} dr\Big).$$

Derivando e tomando e tomando o negativo da função de partição,

$$- \frac{\partial ln Z}{\partial \beta} = \frac{3N}{2} \frac{1}{\beta} + \frac{\int_{r} U(r) e^{-\beta U(r)} dr}{\int_{r} e^{-\beta U(r)} dr},$$

sendo $\beta = 1/k_b T$, então a energia média é:

$$ \langle E \rangle = \frac{3N}{2} k_b T + \frac{\int_{r} U(r) e^{-\beta U(r)} dr}{\int_{r} e^{-\beta U(r)} dr}.$$

E o termo que estamos interessados em calcular com a simulação é a integral:

$$ \frac{\int_{r} U(r) e^{-\beta U(r)} dr}{\int_{r} e^{-\beta U(r)} dr}.$$

Como o problema deseja que a solução seja a partir de uma distribuição uniforme, então temos:

$$ \frac{\langle U(\eta) e^{-\beta U(\eta)} \rangle_{n}}{\langle e^{-\beta U(\eta)} \rangle_{n}}$$

em que $\eta$ são as coordenadas aleatórias geradas e $n$ o número de configurações. O valor médio da integral que envolve as interações não ligadas é:

$$\langle U(\eta) e^{-\beta U(\eta)} \rangle_{n} = \frac{1}{n} \sum_{i} U(\eta_{i}) e^{-\beta U(\eta_i)}$$

$$\langle e^{-\beta U(\eta)} \rangle_{n} = \frac{1}{n} \sum_{i} e^{-\beta U(\eta_i)}$$


$$\frac{\sum_{i} U(\eta_{i}) e^{-\beta U(\eta_i)}}{ \sum_{i} e^{-\beta U(\eta_i)}}.$$

programando essa quinta parte.


In [ ]:
import numpy as np



def media_numerador(u):
    '''
    Essa função recebe a energia potencial
    de interação entre as particulas do
    sistema (U(r)) e retorna o produto
    entre esse potencial e o exponecial do
    potencial.
    
    Unidades de Medida (entrada)
    ------------------
    potencial(U)       [kcal/mol]
    
    Unidades de Medida (saida/retorno)
    ------------------
    media              [kcal/mol]
    
    Unidades de Constantes
    ------------------
    Constante de Boltzmann(kb) [kcal/K mol]
    Temperatura(T)             [K]     
    '''
    
    kb = 0.0019851
    T = 90
    beta = 1/(kb*T)
    
    return u*np.exp(-beta*u)
    

def media_denominador(u):
    '''
    Essa função recebe a energia potencial
    de interação entre as particulas do
    sistema (U(r)) e retorna o exponecial 
    do potencial.
    
    Unidades de Medida (entrada)
    ------------------
    potencial(U)       [kcal/mol]
    
    Unidades de Medida (saida/retorno)
    ------------------
    media              [kcal/mol]
    
    Unidades de Constantes
    ------------------
    Constante de Boltzmann(kb) [kcal/K mol]
    Temperatura(T)             [K]     
    '''
    
    kb = 0.0019851
    T = 90
    beta = 1/(kb*T)
    
    return np.exp(-beta*u)


numero_de_atomos = 100
massa_molar = 39.963
densidade = 1.394

lado = lado_cubo(numeros_de_atomos, massa_molar, densidade)

epsilon =  0.2378
sigma = 3.41

configuracoes = [10, 10**2, 10**3, 10**4, 10**5, 10**6, 10**7, 10**7]


for conf in configuracoes:    
    for ni in range(conf+1):
        U_positivo = 0
        U_total = 0
        media_tot_numerador = 0
        media_tot_denominador = 0
        for i in range(numero_de_atomos):
            r = coords_x_y_z(lado)
            U = potencial(r, epsilon, sigma)
            U_total += U
            media_tot_numerador += media_numerador(U_total)
            media_tot_denominador += media_denominador(U_total)
    
    E_media = round(media_tot_numerador/media_tot_denominador, 6)         
    #print(media_tot_numerador)
    #print(media_tot_denominador)
    print(f'Energia Potencial Média: {E_media} kcal/mol, para {ni} Configurações.')
    
     
      
 

Energia Potencial Média: -0.385892 kcal/mol, para 10 Configurações.
Energia Potencial Média: 0.002846 kcal/mol, para 100 Configurações.
Energia Potencial Média: -0.529206 kcal/mol, para 1000 Configurações.
Energia Potencial Média: -0.75793 kcal/mol, para 10000 Configurações.
Energia Potencial Média: -0.005381 kcal/mol, para 100000 Configurações.
